<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span><ul class="toc-item"><li><span><a href="#загрузка-и-изучение-данных" data-toc-modified-id="загрузка-и-изучение-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>загрузка и изучение данных</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Предобработка данных</a></span></li></ul></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span><ul class="toc-item"><li><span><a href="#Преобразование-данных" data-toc-modified-id="Преобразование-данных-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Преобразование данных</a></span></li><li><span><a href="#Проверка-работоспособности-алгоритма" data-toc-modified-id="Проверка-работоспособности-алгоритма-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Проверка работоспособности алгоритма</a></span></li></ul></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

В процессе выполнения проекта мы выполним следующие этапы:
- загрузим и изучим файл
- проведем предобработку данных (если это потребуется)
- ответим на вопрос изменится ли качество линейной регресси при перемножении признаков на обратную матрицу или нет
- предложим алгоритм преобразования данных для решения основной задачи
- проверим качество линейной регресси на приобразованных данных и посчитаем коэффициент детерминации

## Загрузка данных

### загрузка и изучение данных

Перед началом работы загрузим все необходимые библиотеки.

In [1]:
import pandas as pd
import numpy as np
#import random

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

Прочитаем данные из файла.

In [2]:
insurance = pd.read_csv('/datasets/insurance.csv') # открытие файла

In [3]:
display(insurance.head()) # просмотр первых пяти строк.

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Сразу можно сказать, что названия столбцов мы заменим для удобства дальнейшей работы. Далее посмотрим информацию о файле.

In [4]:
insurance.info() # просмотр информации о файле

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


В файле 5 столбцов: из них в двух тип -float64, а у трех- int64. Тип данных в столбце с возрасnом и зарплатой мы заменим на целочисленный.

В соответствии с документацией в таблице присутствуют следующие данные:
- пол
- возраст
- зарплата застрахованного
- количество членов его семьи
- количество страховых выплат клиенту за последние 5 лет.


Проверим есть ли дубликаты строк.

In [5]:
insurance.duplicated().sum() # подсчет количества дублирующихся строк

153

153 дублирующиеся строки. Поскольку у нас нет какого-то идентификатора клиента (его ФИО или его ID), мы не можем сказать, что дублирующиеся строки описывают одних и тех же застрахованных. В случае, если бы в таблице были бы ID людей и обнаружились бы дубли, то в таком случае мы бы однозначно удалили бы такие строки. В нашем случае у нас нет информации о том, что это дейтвительно задвоенная информация об одних и тех же людях, поэтому дубли удалять не будем.

Посмотрим как коррелируют между собой данные.

In [6]:
insurance.corr() # расчет попарного коэффициента корреляции

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
Пол,1.000000,0.002074,0.014910,-0.008991,0.010140
Возраст,0.002074,1.000000,-0.019093,-0.006692,0.651030
Зарплата,0.014910,-0.019093,1.000000,-0.030296,-0.014963
Члены семьи,-0.008991,-0.006692,-0.030296,1.000000,-0.036290
Страховые выплаты,0.010140,0.651030,-0.014963,-0.036290,1.000000


Между признаками везде слабая корреляция. Достаточно сильная корреляция наблюдается между признаком "возраст" и целевым признаком "страховые выплаты". Далее посмотрим статистические данные о признаках.

In [7]:
insurance[['Возраст', 'Зарплата', 'Члены семьи', 'Страховые выплаты']].describe()

,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000
mean,30.952800,39916.360000,1.194200,0.148000
std,8.440807,9900.083569,1.091387,0.463183
min,18.000000,5300.000000,0.000000,0.000000
25%,24.000000,33300.000000,0.000000,0.000000
50%,30.000000,40200.000000,1.000000,0.000000
75%,37.000000,46600.000000,2.000000,0.000000
max,65.000000,79000.000000,6.000000,5.000000


Видимых проблем в данных нет. Проверим, что у нас в графе "пол" нет никаких неожиданностей.

In [8]:
insurance['Пол'].unique()

array([1, 0])

**Вывод**

Мы посмотрели данные, критических ошибок и аномалий не обнаружено, но нужно провести некоторые преобразования, для удобства дальнейшей работы:
- заменим названия в столбцах на английские
- изменим тип данных в столбце с возрастом и зарплатой


### Предобработка данных

Переименовываем столбцы:

In [9]:
insurance.columns = ['gender', 'age', 'income', 'family_members', 'insurance_payments']

Заменяем тип данных с вещественного на целочисленный:

In [10]:
insurance[['age', 'income']] = insurance[['age', 'income']].astype('int64')

Проверяем результат преобразований:

In [11]:
display(insurance.head())

,gender,age,income,family_members,insurance_payments
0,1,41,49600,1,0
1,0,46,38000,1,1
2,0,29,21000,0,0
3,0,21,41700,2,0
4,1,28,26100,0,0


## Умножение матриц

Обозначения:
- $a_1$, $a_2$- предсказание моли на оригинальной и обратимой матрицах

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w_1$, $w_2$  — вектор весов линейной регрессии (нулевой элемент равен сдвигу)
- $E$ - единичная матрица
- $A$ и $B$- произвольные квадратные матрицы


Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Качество предсказаний моделей не изменится.

**Обоснование:** 

Необходимо доказать, что $$a_1 = a_2$$

Распишем более подробно чему равны предсказания:
$$
a_1 = Xw_1 = XEw_1 = XPP^{-1}w_1
$$
$$
a_2 = Xw_2 = XPw_2
$$

Таким образом получаем:
$$a_1 = a_2$$
$$XPP^{-1}w_1 = XPw_2$$
$$P^{-1}w_1 = w_2$$



Далее исходим из того, что X и P не нулевые матрицы.

Распишем чему равен вектор весов в каждом случае:
$$
w_1 = (X^T X)^{-1} X^T y
$$

$$
w_2 = ((XP)^T XP)^{-1} (XP)^T y
$$

Подставим их в равенство:

$$
P^{-1}(X^T X)^{-1} X^T y = ((XP)^T XP)^{-1} (XP)^T y
$$


Воспользуемся свойствами для двух квадратных матриц $(AB)^T = B^TA^T$ и $(AB)^{-1} = B^{-1}A^{-1}$

$$
P^{-1}(X^T X)^{-1} X^T y = (X^TP^T XP)^{-1} X^TP^T y
$$

$$
P^{-1}(X^T X)^{-1} X^T y = (X^T XP)^{-1}(P^T)^{-1}P^T X^T y
$$

$$
P^{-1}(X^T X)^{-1} X^T y = P^{-1}(X^T X)^{-1} X^T y
$$

Обе части уравнения равны, соответственно:

$$a_1 = a_2$$
$$XPP^{-1}w_1 = XPw_2$$
$$MSE(a_1, y) = MSE(a_2, y)$$

Мы доказали, что предсказания модели на оригинальной и преобразованной выборках будет одинаковым и следовательно качество предсказаний не изменится.

## Алгоритм преобразования

**Алгоритм**

Для того, чтобы зашифровать данные клиентов страховой компании, мы прибегнем к достаточно простому методу: матрицу признаков домножим на случайную обратимую квадратную матрицу, которую сгенерируем случайным образом.

Наш алгоритм будет состоять из следующих операций:
- Для начала мы создадим функцию, которая будет генерировать случайную матрицу нужного размера и проверять ее на обратимость.
- Далее создадим еще две функции: одна для кодирования признаков, а другая для декодирования. 
- Разделим исходные данные на признаки и целевой признак. 
- Признаки преобразуем с помощью функции кодирования и сразу проверим, что их можно декодировать. 
- Далее разделим данные на обучающую и тестовые выборки. 
- Обучим модели линейной регрессии на оригинальных и измененных данный и посмотрим меняется ли коэффициент детерминации. 

**Обоснование**

Во втором разделе мы доказали, что при предсказания модели на оригинальной и преобразованной выборках будет одинаковым. 

Формула обучения будет выглядеть следующим образом:
$$
a_2 = XPw_2 = XP((XP)^TXP)^{-1}(XP)^Ty
$$

## Проверка алгоритма

### Преобразование данных

Напишем несколько функций:
- matrix_generators - для генерации случайных обратимых матриц
- matrix_encoder - для шифрования исходных данных
- matrix_decoder - для шифрования исходных данных

In [12]:
def matrix_generator(shape, max_iterations=10): #функция для генерации случайной матрицы
    matrix_generated1 =np.random.default_rng()
    matrix_generated = matrix_generated1.integers(0, 100, (shape))
    
    for _ in range(max_iterations):
        
        
        try:
            matrix_inverse = np.linalg.inv(matrix_generated)
            
            return matrix_generated, matrix_inverse
          
        
        except np.linalg.LinAlgError:
            pass
        

                          
                           
                           
        

In [13]:
def matrix_encoder(features, max_iterations=10): # функция кодирования
    encryption_key, decryption_key = matrix_generator((features.shape[1], features.shape[1]), max_iterations)
      
    return features.dot(encryption_key), decryption_key
    


In [14]:
def matrix_decoder(features, decryption_key): # функция декодирования
       
    return features.dot(decryption_key)

Далее мы применим наши функции и проверим качество моделей на оригинальных признаках и закодированных.

### Проверка работоспособности алгоритма

Разделим нашу выборку на признаки и целевой признак.

In [15]:
target_original = insurance['insurance_payments'] # целевой признак
features_original = insurance.drop('insurance_payments', axis=1) # признаки

Оригинальные данные разделим на обучающую и тестовую выборки и проверим, что признаки и целевой признак разделены верно.

In [16]:
features_train_original, features_test_original, target_train, target_test = train_test_split(features_original,
                                                            target_original, test_size=0.25, random_state=12345)

In [17]:
print(features_train_original.shape)
print(target_train.shape)

(3750, 4)
(3750,)


In [18]:
print(features_test_original.shape)
print(target_test.shape)

(1250, 4)
(1250,)


Зашифруем признаки с помощью нашей функции для кодирования

In [19]:
features_original_matrix = features_original.values # преобразуем признаки в матрицу
features_encoder, decrypter = matrix_encoder(features_original_matrix) # применим функцию для кодирования


Сразу проверим, что данные правильно декодируются.

In [20]:
features_decrypted = matrix_decoder(features_encoder, decrypter) # применим функцию для декодирования

In [21]:
print(features_original_matrix) # просмотр оригинальной матрицы
print(features_decrypted) # просмотр декодированной матрицы
print(features_original_matrix == (np.around([features_decrypted], decimals=0))) #сравнение оригинальной и декодированной матрицы

[[    1    41 49600     1]
 [    0    46 38000     1]
 [    0    29 21000     0]
 ...
 [    0    20 33900     2]
 [    1    22 32700     3]
 [    1    28 40600     1]]
[[ 1.00000000e+00  4.10000000e+01  4.96000000e+04  1.00000000e+00]
 [-2.33299144e-12  4.60000000e+01  3.80000000e+04  1.00000000e+00]
 [-1.91738986e-13  2.90000000e+01  2.10000000e+04  4.19175111e-13]
 ...
 [-5.07447556e-12  2.00000000e+01  3.39000000e+04  2.00000000e+00]
 [ 1.00000000e+00  2.20000000e+01  3.27000000e+04  3.00000000e+00]
 [ 1.00000000e+00  2.80000000e+01  4.06000000e+04  1.00000000e+00]]
[[[ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]
  ...
  [ True  True  True  True]
  [ True  True  True  True]
  [ True  True  True  True]]]


Декодирование прошло успешно. Теперь преобразуем нашу кодированную матрицу в таблицу.

In [22]:
features_encoder = pd.DataFrame(features_encoder) #переводим  кодированную матрицу в DataFrame

Разделим кодированные признаки на обучающую и тестовую выборки.

In [23]:
features_train_encoder , features_test_encoder = train_test_split(features_encoder, random_state=12345)

Обучим две модели линейной регрессии на оригинальных признаках и на закодированных.

In [24]:
model_0 = LinearRegression() # инициализация модели линейной регрессии
model_0.fit(features_train_original, target_train) # обучение модели
predicted_test_0 = model_0.predict(features_test_original) # получение предсказания результата на тестовой выборке

In [25]:
model_1 = LinearRegression() # инициализация модели линейной регрессии
model_1.fit(features_train_encoder, target_train) # обучение модели
predicted_test_1 = model_1.predict(features_test_encoder) # получение предсказания результата на тестовой выборке

Посчитаем метрику R2 для обеих моделей.

In [26]:
clf1_r2 = r2_score(target_test, predicted_test_0) # расчет коэфициента детирминации
clf2_r2 = r2_score(target_test, predicted_test_1) 
print(f'R2 score на исходных данных: {clf1_r2:.5f}') 
print(f'R2 score на преобразованных данных: {clf2_r2:.5f}')

R2 score на исходных данных: 0.43523
R2 score на преобразованных данных: 0.43523


Коэффициенты детерминации в обоих случаях одинаковые, что подтверждает нашу гипотезу о том, что при умножении матрицы признаков на случайную обратимую квадратную матрицу, качество модели не меняется.

## Вывод

В рамках данного проекта нам нужно было решить задачу по шифрованию данных клиентов страховой компании, а также определить влияет ли умножение признаков на обратимую матрицу на качество модели или нет.

В первом разделе проекта мы посмотрели и оценили качество предоставленных данных. Критических ошибок не обнаружено, пропусков в данных нет. Есть небольшое количество дубликатов, но из-за нехватки информации об ID застрахованного, эти строки нельзя однозначно назвать дубликатами, поэтому мы их не удаляли. Несмотря, на то что ошибок в данных не было, мы все же провели небольшую предобработку для удобства дальнейшей работы: заменили названия столбцов и поменяли тип данных в столбцах с возрастом и зарплатой.

Во втором разделе проекта мы дали ответ на вопрос: "изменится ли качество линейной регрессии, если признаки умножить на обратимую матрицу". Путем последовательных математических вычислений, мы доказали, что качество не изменится.

В третьем разделе мы описали алгоритм шифрования данных.

В последнем разделе проекта, предложенный алгоритм был запрограммирован и проверен. Качество линейной регрессии, как и было доказано ранее, не изменилось - метрики качества R2 одинаковые для обеих моделей (построенной на неизмененных данных и построенной на закодированных данных).

Думаю, что подобный метод шифрования данных имеет право на жизнь, но нужно проверять его на криптографическую устойчивость.